
... ***CURRENTLY UNDER DEVELOPMENT*** ...


# Prepara fichero para Alba R

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.climate_emulator import Climate_Emulator
from teslakit.waves import AWL, Aggregate_WavesFamilies

from teslakit.plotting.outputs import Plot_FitSim_Histograms
from teslakit.plotting.extremes import Plot_FitSim_AnnualMax, Plot_FitSim_GevFit, Plot_Fit_QQ

from teslakit.plotting.waves import Plot_Waves_Histogram_FitSim


In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'

# climate change - S5
site = 'KWAJALEIN_CC_S5'

db_S5 = Database(p_data)
db_S5.SetSite(site)

## Load waves from the climate emulator

In [3]:
# Full simulation

CE = Climate_Emulator(db_S5.paths.site.EXTREMES.climate_emulator)

WVS_upd = CE.LoadSim_All(TCs=True)
WVS_upd = WVS_upd.to_xarray()

print(WVS_upd)

<xarray.Dataset>
Dimensions:      (index: 1030828)
Coordinates:
  * index        (index) int64 0 1 2 3 4 ... 1030824 1030825 1030826 1030827
Data variables:
    DWT          (index) float32 28.0 1.0 32.0 40.0 32.0 ... 39.0 33.0 32.0 32.0
    sea_Hs       (index) float32 2.980855 2.3911204 ... 3.2097461 4.08398
    sea_Tp       (index) float32 8.070791 10.645989 9.094906 ... 9.195859 8.9872
    sea_Dir      (index) float32 52.277348 50.070602 ... 67.18341 77.438225
    swell_1_Hs   (index) float32 0.23376392 1.2217002 ... 0.37393954 0.57650656
    swell_1_Tp   (index) float32 11.979431 14.376366 ... 10.96254 8.847657
    swell_1_Dir  (index) float32 79.708595 78.30079 ... 86.124 52.500004
    swell_2_Hs   (index) float32 0.34879413 0.267301 ... 0.28582856 0.24780126
    swell_2_Tp   (index) float32 12.225544 12.518263 ... 14.174422 11.8886385
    swell_2_Dir  (index) float32 185.89203 163.95964 ... 177.3409 172.38188
    wind_speed   (index) float32 6.572572 7.04338 8.962964 ... 6.99932

In [4]:
#----------------------------------------------------------
# Poner NaN en WVS_upd_E donde Hs, Tp y Dir de Swell_1 y Swell_2 valen 0 (Hubo TC y sólo tenemos Sea family)

ind = np.where(WVS_upd.swell_1_Tp==0.0)[0]

vars = {'swell_1_Hs','swell_1_Tp','swell_1_Dir','swell_2_Hs','swell_2_Tp','swell_2_Dir'}
for var in vars:
    WVS_upd[var][ind] = np.nan


## Separate TC waves from RMC

In [5]:
WVS_sim_TCs = WVS_upd.copy(deep=True)
WVS_sim_RCM = WVS_upd.copy(deep=True)

WVS_sim_TCs = WVS_sim_TCs.where(np.isnan(WVS_sim_TCs.swell_1_Hs), drop=True)
print(WVS_sim_TCs)
print()

WVS_sim_RCM = WVS_sim_RCM.where(~np.isnan(WVS_sim_RCM.swell_1_Hs), drop=True)
print(WVS_sim_RCM)
print()

# Remove time variable
WVS_sim_RCM = WVS_sim_RCM.drop('time')
WVS_sim_TCs = WVS_sim_TCs.drop('time')

# save
from teslakit.io.aux_nc import StoreBugXdset 

StoreBugXdset(WVS_sim_RCM, '/Users/albacid/Projects/TeslaKit_projects/sites/' + site + '/EXTREMES/CE_all_sims_RCM.nc')  
StoreBugXdset(WVS_sim_TCs, '/Users/albacid/Projects/TeslaKit_projects/sites/' + site + '/EXTREMES/CE_all_sims_TCs.nc')  



<xarray.Dataset>
Dimensions:      (index: 11207)
Coordinates:
  * index        (index) int64 45 150 164 192 ... 1030676 1030791 1030817
Data variables:
    DWT          (index) float32 37.0 38.0 39.0 38.0 ... 41.0 42.0 37.0 41.0
    sea_Hs       (index) float32 0.01648 2.4367867 ... 1.2825347 2.0451221
    sea_Tp       (index) float32 1.941 7.8854766 ... 6.7418084 7.090463
    sea_Dir      (index) float32 238.5 125.26454 ... 225.42981 127.9396
    swell_1_Hs   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_1_Tp   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_1_Dir  (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_2_Hs   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_2_Tp   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_2_Dir  (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    wind_speed   (index) float32 2.0832067 12.898403 ... 7.7907186